# DS<sup>2</sup> Probability, Statistics and Bayesian Statistics : Python 실습

TA : 김당찬(dang112@snu.ac.kr)

## 모듈 소개  

1. `scipy.stats`

- `scipy` 패키지는 수치해석, 통계, 신호처리 등 다양한 수학적 기능을 제공하는 패키지입니다. `scipy` 패키지 안의 `stats` 모듈은 확률분포와 통계에 관련된 함수들을 제공합니다.
- `scipy.stats` 패키지 안의 함수로 여러가지 분포함수들을 사용할 수 있습니다. (`binom`, `uniform`, `norm`, `chi2`, `t`, `f` 등)   
  
2. `numpy`

- 벡터, 행렬 등의 표현 및 빠른 계산을 위해서 `numpy` 패키지가 필요합니다. 
- 강의에는 주로 수의 제곱근형태(sqrt), 수열(arange), 나열(array),올림(ceil) 이나 집단의 평균(mean), 분산(var), 표준편차(std) 등을 구하는 데에 주로 사용합니다.
  
3. `matplotlib.pyplot`

- `matplotlib` 패키지는 시각화를 위해 사용되는 파이썬의 대표적인 라이브러리입니다. 
- `matplotlib.pyplot` 모듈은 그래프를 그리기 위한 다양한 함수들을 제공합니다.

`matplotlib.pyplot` 모듈의 함수들 중 자주 사용되는 함수들은 다음과 같습니다.

|     함수     | 설명 |
|----------|-----------------|
|`figure()` | matplotlib 에서 그래프가 들어가는 영역을 의미한다.| 
|`plot()`   |그래프를 그려준다.           | 
|`subplots()` |subplot들을 포함하는 figure를 생성해준다.            |
|`legend()`|범례를 그려준다.|
|`show()`|그래프를 화면에 보여준다.|
|`xlabel()`|x축에 라벨을 출력한다.|
|`ylabel()`|y축에 라벨을 출력한다.|
|`title()`|타이틀을 출력한다.|
|`xlim()`|x축의 상한/하한 값을 설정한다.|
|`ylim()`|y축의 상한/하한 값을 설정한다.|
|`subplot()`|subplot을 지정한다.|

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scienceplots

plt.rcParams['figure.facecolor'] = 'white'
plt.style.use('science')

import scipy
from scipy import stats
from scipy.stats import uniform, norm, invgamma

import arviz as az
import pymc as pm

In [ ]:
# Package versions
print('Running on Python v{}'.format(sys.version))
print('Running on PyMC3 v{}'.format(pm.__version__))
print('Running on ArviZ v{}'.format(az.__version__))
print('Running on NumPy v{}'.format(np.__version__))
print('Running on Pandas v{}'.format(pd.__version__))
print('Running on SciPy v{}'.format(scipy.__version__))
print('Running on Seaborn v{}'.format(sns.__version__))
print('Running on Matplotlib v{}'.format(plt.matplotlib.__version__))

# 표본추출 (Sampling)

• 모집단 (population): 정보를 얻고자 하는 대상이 되는 집단 전체

• 표본 (Sample): 모집단에서 추출한 부분집합

▷ 유한모집단에서 랜덤표본 (Random Sample): 단순 랜덤 비복원추출로 뽑은 표본

e.g.

$$ X_1, X_2, \cdots, X_n \in \{1, 2, \cdots, N\} $$

▷ 무한모집단에서 랜덤표본: 동일한 분포(모집단의 분포)를 따르는 독립인 확률변수들

e.g.

$$ X_1, X_2, \cdots, X_n \sim F $$

### 일반적인 분포로부터의 난수 생성

- `scipy.stats` 모듈이나 `np.random` 모듈을 이용하여 다양한 분포로부터 난수를 생성할 수 있습니다.


#### 균등분포
다음은 균일분포 $U(0, 1)$에서 크기가 10인 표본을 추출하는 코드입니다.

In [ ]:
np.random.seed(0)
uniform.rvs(0,1, 10)

In [ ]:
# Numpy
np.random.seed(0)
np.random.uniform(0,1,10)

#### 정규분포
다음은 정규분포 $N(0, 1)$에서 크기가 10인 표본을 추출하는 코드입니다.

In [ ]:
np.random.seed(0)
norm.rvs(0,1,10)

In [ ]:
np.random.seed(0)
np.random.normal(0,1,10)

#### 난수의 히스토그램
표준정규분포 $N(0, 1)$에서 크기가 $10000$인 표본을 추출해 히스토그램을 그려보면 다음과 같습니다.

In [ ]:
plt.figure(figsize=(6, 6))
np.random.seed(0)
norm_rv = norm.rvs(0,1, 10000)
plt.hist(norm_rv, 100)
plt.show()

확률밀도함수(pdf)와 히스토그램을 비교해보면 다음과 같습니다.

In [ ]:
np.random.seed(0)

fig, ax = plt.subplots(1, 1, figsize=(6, 6))

xs = np.linspace(-4, 4, 1000)

ax.plot(xs, norm.pdf(xs), lw=1.0, label='norm pdf')
ax.hist(norm_rv, bins=100, density=True, alpha=0.5, label='norm rvs')
ax.legend()
plt.show()

## Sampling

### 기각 샘플링 (Rejection Sampling)

`scipy, numpy` 등의 패키지에 내장된 대표적인 확률분포로부터 샘플링 하는 것은 간단합니다. 그러나 다음과 같이 임의로 설정된 복잡한 확률분포에 대한 샘플링 문제를 해결하기 위해서는 다른 방법을 이용해야 합니다.

실습자료에 예시로 제시된
$$
f(x)=0.3 \exp \left(-0.1 x^2\right)+0.7 \exp \left(-0.2(x-10)^2\right)
$$
를 따르는 $X$를 샘플링하는 문제를 생각해보겠습니다.

이러한 경우, rejection sampling을 이용하여 샘플링을 시행할 수 있습니다.

제안분포로 사용될 $g$는 다음과 같습니다. ($f$보다 난수생성이 쉬운 분포 e.g. Uniform distribution)
$$
g(X) = \frac{1}{24}(-7<X<17) \sim \text{Unif}(-7,17)
$$

In [ ]:
# 확률밀도함수 그리기 

def f(x):
    return 0.3*(np.exp(-0.2*x**2)) + 0.7*(np.exp(-0.2*(x-10)**2))

xs = np.linspace(-10, 20, 10000)
fxs = f(xs)

plt.figure(figsize=(6, 6))
plt.plot(xs, fxs, label="PDF")
plt.legend()
plt.show()

In [ ]:
# 제안분포 그리기
def g(x):
    return uniform(loc=-7, scale=24).pdf(x)

plt.figure(figsize=(6, 6))
plt.plot(xs, fxs, label="Target")
plt.plot(xs, g(xs), label="Proposal")
plt.legend()
plt.show()

$\beta=\dfrac{1}{24*0.7}$으로 설정하면, 다음과 같이 upper envelope를 구할 수 있다.

In [ ]:
beta = 1 / (24 * 0.7)

plt.figure(figsize=(6, 6))
plt.plot(xs, fxs, label="Target")
plt.plot(xs, g(xs) / beta, label="Proposal")
plt.ylim(0,1.)
plt.legend(loc="upper left")
plt.show()

이제 Rejection sampling 일고리즘으로부터 난수를 생성해보자.

(1) $g$로부터 $X$를 생성한다.

(2) $U(0, 1)$로 부터 $U$ 를 생성한다.

(3) 만약 $Ug(X) ≤ βf(X)$를 만족하면 $X$를 $f$ 에서 생성된 난수라고 받아들인다. 만족하지 않으면 아니라고 받아들인다

In [ ]:
n = 10000 # 반복 횟수

# Step 1 : g로부터 샘플 생성

Xs = uniform.rvs(-7, 24, n)

In [ ]:
# Step 2
Us = uniform.rvs(0,1, n)

In [ ]:
# Step 3
accept = np.where(g(Xs) * Us < beta * f(Xs))

samples = Xs[accept]

In [ ]:
# Plot
fig, ax = plt.subplots(1,1, figsize=(6, 6))

ax.hist(samples, bins=50, density=True, label='sample')
ax.plot(xs, fxs, label='true')
ax.legend()
plt.show()

In [ ]:
from scipy.integrate import quad

quad(f, -np.inf, np.inf)[0]

In [ ]:
# Plot after normalization

fig, ax = plt.subplots(1,1, figsize=(6, 6))

ax.hist(samples, bins=50, density=True, label='sample')
ax.plot(xs, fxs / quad(f, -np.inf, np.inf)[0], label='true')
ax.legend()
plt.show()

Note : `np.random.seed()` 등 random number seed를 고정시키고자 할 때, 각각의 $X,U$ 등을 생성할 때마다 seed를 고정시키게 되면 두 변수의 상관관계가 생길 수 있음에 주의하자.

In [ ]:
np.random.seed(0)
Xs = uniform.rvs(-7, 24, n)

np.random.seed(0)
Us = uniform.rvs(0,1, n)

In [ ]:
# Step 2
accept = np.where(g(Xs) * Us < beta * f(Xs))
samples = Xs[accept]

# Plot
fig, ax = plt.subplots(1, 1, figsize=(6, 6))
ax.hist(samples, bins=50, density=True, label='sample')
ax.plot(xs, fxs / quad(f, -7, 17)[0], label='true')
ax.legend()
plt.show()

# Markov Chain Monte Carlo

## Metropolis-Hastings

### 목적 
- 원하는 분포 $P(x)$ 에서 direct sampling이 어려울 때, 그 분포를 따르는 random sample을 얻기 위해서 시행하는 알고리즘
- Approximate the desired distribution (histogram)
- Compute an integral (ex : 기댓값 계산)

### Algorithm

1. 초기값 설정 $x_0$, $\;t=0$

2. 제안분포(or jumping kernel) $g(\cdot)$에서 다음 단계의 후보값 $x^{\prime}$을 랜덤하게 뽑는다.

3. 다음 **Acceptance probability**를 계산한다.

$$
\left(A\left(x^{\prime}, x_{t}\right)=\min \left(1, \frac{P\left(x^{\prime}\right)}{P\left(x_{t}\right)} \frac{g\left(x_{t} | x^{\prime}\right)}{g\left(x^{\prime} | x_{t}\right)}\right) \right)
$$

4. Accept of Reject 결정 : $x^{t+1} = x^{\prime}$ with probability $A(x^{\prime}, x_t)$

$$
\begin{array}{l}
{\text { [1] generate a uniform random number } u \in[0,1] \text { ; }} \\ 
{\text { [2] if } u \leq A\left(x^{\prime}, x_{t}\right), \text { accept the new state and set } x_{t+1}=x^{\prime} \text { ; }} \\ 
{\text { [3] if } u>A\left(x^{\prime}, x_{t}\right), \text { reject the new state, and copy the old state forward } x_{t+1}=x_{t} \text { ; }}\end{array}
$$

5. Set $t = t+1$, step 2-5를 반복한다.

- 다음 상태로 나아갈지에 대한 여부를 acceptance probability에 근거하여 결정하는 알고리즘  

> Sometimes accepting the moves and sometimes remaining in place

- Acceptance ratio $\alpha$의 의미 : 새로운 후보값이 현재값에 비해 $P(x)$에서 나온 sample일 가능성 (>1 : 후보값이 현재값보다 원하는 분포에서 나왔을 가능성이 더 높다.) 

- 그러므로, 단계가 반복될수록 $P(x)$의 high-density regions에서 sample을 뽑게 된다. 

### Previous Example

- 이전 기각 샘플링 예시에서 $(-7,17)$ 외 영역에서의 샘플링은 이루어지지 않았습니다.
$$
f(x)=0.3 \exp \left(-0.2 x^2\right)+0.7 \exp \left(-0.2(x-10)^2\right)
$$

- 이를 Metropolis-Hastings 알고리즘을 이용하여 샘플링해 보겠습니다.
- 제안분포로는 정규분포를 사용합니다.
$$g(x^\prime|x_t) = N(x_t, 1^2)$$

In [ ]:
# Metropolis-Hastings

def MH(seed, n):
    np.random.seed(seed)

    x = np.zeros(n)
    x[0] = 5 # 초기값

    for i in range(1, n):
        z = norm.rvs(x[i-1], 1) # 제안분포
        r = min(1, f(z) / f(x[i-1])) # acceptance ratio
        if uniform.rvs(0,1) < r:
            x[i] = z
        else:
            x[i] = x[i-1]

    return x

x = MH(30, 100000)
fig, ax = plt.subplots(1, 1, figsize=(6, 6))
ax.plot(x)
ax.set_title('MH of $f(x) = 0.3N(0,2.5) + 0.7N(10,2.5)$')
plt.show()

In [ ]:
# Burn-in 제외, histogram plot

fig, ax = plt.subplots(1, 1, figsize=(6, 6))
ax.hist(x[10000:], bins=50, density=True, label='sample')
ax.plot(xs, fxs / quad(f, -7, 17)[0], label='true')
ax.legend()
plt.show()

MCMC는 seed 설정에 의해 Markov chain이 생성되므로, sample이 충분하지 않다면 seed를 변화시킴에 따라 결과가 달라질 수 있습니다.

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

for i, seed in enumerate([0, 10, 20]):
    x = MH(seed, 10000)
    # trace plot
    axes[0, i].plot(x)
    axes[0, i].set_title('seed = {}'.format(seed))
    # histogram
    axes[1, i].hist(x[9000:], bins=50, density=True, label='sample')
    axes[1, i].plot(xs, fxs / quad(f, -np.inf, np.inf)[0], label='true')
    axes[1, i].legend()

plt.tight_layout()
plt.show()

### 예시 1

어느 공정에 하루동안 투입되는 화학물질의 양의 100일간 데이터가 있다. 이 데이터는 정규분포를 따르고, 과거 정보에 의하면 표준편차가 1이라고 알려져있다. 이 때 화학물질이 투입되는 일일 평균양에 대한 사후분포를 M-H 방법으로 구해보자.



- $Y_1, ...,Y_{100} \sim i.i.d \; N(\theta,1)$의 데이터가 존재  ($\theta = 3$)


- $\theta$의 사전분포 :
$$\pi_0(\theta)=\frac{1}{\pi(1+\theta^2)}$$


- 사후분포 : 
$$
\begin{aligned} \pi\left(\theta | Y_{1}, \cdots, Y_{n}\right) \; & \propto \exp \left(-\frac{\sum_{i=1}^{n}\left(Y_{i}-\theta\right)^{2}}{2}\right) \times \frac{1}{1+\theta^{2}} \\ & \propto \exp \left(-\frac{n(\theta-\overline{Y})^{2}}{2}\right) \times \frac{1}{1+\theta^{2}} \end{aligned}
$$



- 제안분포 : 
$$q\left(\theta | \theta^{*}\right)=\frac{1}{\sqrt{2 \pi n^{-1}}} e^{-\frac{(\theta-\overline{Y})^{2}}{2 / n}}, \text { i.e. } N\left(\overline{Y}, \frac{1}{n}\right)$$


- acceptance ratio $\alpha$ : 
$$
\begin{aligned} \alpha\left(\theta^{(i-1)}, \tilde{\theta}\right) &=\min \left(1, \frac{\pi\left(\tilde{\theta} | Y_{1}, \cdots, Y_{n}\right) q\left(\theta^{(i-1)} | \tilde{\theta}\right)}{\pi\left(\theta^{(i-1)} | Y_{1}, \cdots, Y_{n}\right) q\left(\tilde{\theta} | \theta^{(i-1)}\right)}\right) \\ &=\min \left(1, \frac{1+\left(\theta^{(i-1)}\right)^{2}}{1+(\tilde{\theta})^{2}}\right) \end{aligned}
$$

In [ ]:
np.random.seed(123)

# Observation of 100 individual, with mean = 3 and scale = 1 (실습에서는 데이터를 generate해서 사용하자.)

n = 100
observation = norm.rvs(3, 1, n)

In [ ]:
# 관측한 data의 histogram
fig, ax = plt.subplots(1,1, figsize=(6,6))
sns.distplot(observation, ax=ax, kde=True, bins=20)
plt.xlabel("Value")
plt.ylabel("Frequency")
plt.title("100 sample from a pop of 10,000 with $\mu=3, \sigma=1$")
plt.show()

In [ ]:
mu_obs = observation.mean()
mu_obs

In [ ]:
# setting
para_init = 1
iterations = 10000

In [ ]:
### Metropolis-Hastings algorithm ###

theta = []
theta_old = para_init

for i in range(iterations):
    theta_new = norm.rvs(mu_obs, 1/n, 1)[0] # 제안분포로부터 샘플링
    log_alpha = min(np.log(1), np.log(1+theta_old**2)-np.log(1+theta_new**2)) # log of acceptance probability
    r = uniform.rvs(0, 1, 1)[0]
    if np.log(r) < log_alpha:
        theta_old = theta_new
        theta.append(theta_new)
    else :
        theta.append(theta_old)

In [ ]:
# output
len(theta)

In [ ]:
# 앞의 9000개는 burn-in period, 나머지 1000개 샘플로 정상분포 추정
theta_burned = theta[9000:]

In [ ]:
# histogram of MCMC samples
plt.figure(figsize=(6, 6))
plt.hist(np.array(theta_burned), bins = 50)
plt.xlabel("Value")
plt.ylabel("Frequency")
plt.title("Histogram of theta")
plt.show()

In [ ]:
# trace plot
plt.figure(figsize=(6, 6))
plt.plot(theta)
plt.xlabel("iter")
plt.ylabel("theta")
plt.title("Markov chain by M-H of theta")
plt.show()

In [ ]:
plt.figure(figsize=(6, 6))
plt.plot(np.linspace(9001, 10000, 1000), np.array(theta_burned))
plt.xlabel("iter")
plt.ylabel("theta")
plt.title("Markov chain by M-H of theta after burn-in period")
plt.show()

In [ ]:
# 사후기댓값의 추정량
print(np.mean(theta_burned))
# 사후중앙값의 추정량
print(np.median(theta_burned))
# credible interval
print(round(np.percentile(theta_burned, 2.5), 4)) # 2.5% 분위수
print(round(np.percentile(theta_burned, 97.5), 4)) #97.5% 분위수

## 깁스 샘플링(Gibbs Sampling)
### 특징
- 2차원 이상의 모수에 대한 사후분포를 각 모수에 대한 조건부 사후분포의 집합으로 나타낸다.

    ex) 2차원 모수 $\Theta=(\mu, \sigma)$에 대한 사후분포 $\pi(\mu, \sigma | Y_{1}, \cdots, Y_{n})$를 $\{\pi(\mu| \sigma, Y_{1}, \cdots, Y_{n}),\; \pi(\sigma | \mu, Y_{1}, \cdots, Y_{n})\}$으로 나타낸다. 

- joint distribution을 구하기 어려울 때나 이 분포에서 바로 sampling하기 어려울 때 각 모수에 대한 conditional distribution을 안다면 적용 가능

- algorithm은 다른 변수의 현재값에 따라 각 변수의 분포에서 차례로 샘플을 생성한다. (즉, 생성된 samplie sequence는 Markov Chain을 구성)

- Markov Chain의 정상분포는 joint distribution (사후분포)

$$ $$

참고) Conditional distribution과 Joint distribution의 관계 

: 한 변수에 대한 conditional distribution(다른 변수들은 고정된 상태)은 joint distribution에 비례한다.

<br>

$$p\left(x_{j} | x_{1}, \ldots, x_{j-1}, x_{j+1}, \ldots, x_{n}\right)=\frac{p\left(x_{1}, \ldots, x_{n}\right)}{p\left(x_{1}, \ldots, x_{j-1}, x_{j+1}, \ldots, x_{n}\right)} \propto p\left(x_{1}, \ldots, x_{n}\right)$$

### Example 2
어떤 공장의 각 Lot별 생산된 wafer들의 수율(%)은 정규분포를 따른다고 한다. 50개의 Lot을 랜덤 추출하여 Lot의 수율에 대한 평균과 분산을 Gibbs Sampling 방법으로 베이지안 추정을 해보자.

- $X_{1}, \cdots, X_{50} \overset{iid}{\sim} N\left(\mu, \sigma^{2}\right) $의 데이터가 존재 ($\mu = 95(\%), \sigma^2 = 1(\%)$)

- 목적 : $\mu, \sigma^2$ 추정하는 것

- 사전분포 : $\pi(\mu) \propto 1 \;\; \& \;\; \pi(\sigma^2) \propto \frac{1}{\sigma^2}$

- 조건부 사후분포 :
 $$\begin{aligned} \mu | \boldsymbol{X}, \sigma^{2} & \sim N\left(\overline{X}, \frac{\sigma^{2}}{n}\right) \\ \sigma^{2} | \boldsymbol{X}, \mu & \sim \text { IGamma }\left(\frac{n}{2}, \frac{1}{2} \sum_{i=1}^{n}\left(X_{i}-\mu\right)^{2}\right) \end{aligned}$$
 
- Algorithm

$$
\displaystyle
\begin{array}{l}
{\text { [1] Initialize } \left( \mu^{(0)}, \sigma^{2(0)}\right)}\\\\
{ \text{For s=1  to N}} : \\\\
{ \text { [2] } \mu^{(s)} \sim N\left(\overline{X}, \frac{\sigma^{2(s-1)}}{n}\right)}\\ 
{ \text{ [3] }  \sigma^{2(s)} \sim \text { InvGamma }\left(\dfrac{n}{2}, \dfrac{1}{2} \sum_{i=1}^{n} \left(X_{i}-\mu^{(s)}\right)^{2}\right)} \end{array}
$$

초기값으로 $\mu^{(0)} = 95\%, \sigma^{2(0)} = 0.3 \%$로 놓고, $N = 10,000$개의 Gibbs sampler $\left(\left(\mu^{(s)}, \sigma^{2(s)}\right), s=1, \cdots, 10000\right)$를 생성

In [ ]:
# 평균 95, 표준편차 1인 정규분포를 따르는 50개의 데이터
n = 50
obs = norm.rvs(95, 1, n)

# 관측한 data의 histogram
plt.figure(figsize = (6, 6))
plt.hist(obs, bins = 10)
plt.xlabel("Value")
plt.ylabel("Frequency")
plt.title("Distribution of 50 obs with mu=95, sigma=1")
plt.show()

In [ ]:
xbar = obs.mean()
xbar

In [ ]:
# setting initial values
mu_init = 95
sigma2_init = 0.3
N = 10000

# conditional distribution 함수 정의
def mu_given_sigma2(s2):
    # s2는 i-1번째 분산 값
    return norm.rvs(xbar, np.sqrt(s2)/np.sqrt(n))

def sigma2_given_mu(m):
    # m은 i번째 update된 mu값
    beta = sum((obs-m)**2)/2
    return invgamma.rvs(a = n/2, scale = beta, size = 1)

In [ ]:
### Gibbs Sampling algorithm ###
mu = []
sigma2 = []
mu_old = mu_init
sigma2_old = sigma2_init

for i in range(N):
    mu_new = mu_given_sigma2(sigma2_old)
    sigma2_new = sigma2_given_mu(mu_new)
    mu_old = mu_new
    sigma2_old = sigma2_new
    mu.append(mu_new)
    sigma2.append(sigma2_new)

In [ ]:
# 앞의 5000개는 burn-in period, 나머지 5000개로 정상분포 추정
mu_burn = mu[5000:] 
sigma2_burn = sigma2[5000:]

plt.figure(figsize = (6, 12))
plt.subplot(211)
plt.hist(np.array(mu_burn), bins = 30, )
plt.xlabel("Value")
plt.ylabel("Frequency")
plt.title("Histogram of mu")

plt.subplot(212)
plt.hist(np.array(sigma2_burn), bins = 30, )
plt.xlabel("Value")
plt.ylabel("Frequency")
plt.title("Histogram of sigma2")
plt.show()

In [ ]:
# trace plot
plt.figure(figsize = (10, 10))
plt.subplot(211)
plt.plot(np.linspace(5001, 10000, 5000), np.array(mu_burn))
plt.xlabel("iter")
plt.ylabel("mu")
plt.title("Markov chain by Gibbs sampling")

plt.subplot(212)
plt.plot(np.linspace(5001, 10000, 5000), np.array(sigma2_burn))
plt.xlabel("iter")
plt.ylabel("sigma2")
plt.show()

In [ ]:
# MAP

print(np.mean(mu_burn))
print(np.mean(sigma2_burn))

### Example 3
고속도로의 부분적 보수에 쓰이는 새로운 시멘트 혼합물이 굳을 때까지 걸리는 시간에 대한 1000번의 측정 자료가 있다. 이 시간 자료는 정규분포를 따른다고 가정할 때, 시간에 대한 평균과 분산을 Gibbs Sampling 방법으로 베이지안 추정을 해보자.


- $X_{1}, \cdots, X_{n=1000} \sim \text { i.i.d } N\left(\mu, \sigma^{2}\right) $의 데이터가 존재 ($\mu = 30(분), \sigma^2 = 25(분)$)

- 목적 : $\mu, \sigma^2$ 추정하는 것

- 사전분포(noninformative) : $\pi(\mu) \sim N(0, 10^2) \;\; \& \;\; \pi(\sigma^2) \sim IGamma(1, 5)$

- 조건부 사후분포 :
 $$\begin{aligned} \mu | \boldsymbol{X}, \sigma^{2} & \sim N\left(\frac{n \overline{Y} \sigma^{-2}}{n \sigma^{-2}+ \frac{1}{100}},\; \frac{1}{n \sigma^{-2}+\frac{1}{100}}\right) \\ \sigma^{2} | \boldsymbol{X}, \mu & \sim \text { InvGamma }\left(\frac{n}{2}+1,\; \frac{1}{2} \sum_{i=1}^{n}\left(X_{i}-\mu\right)^{2}+5\right) \end{aligned}$$
 
- algorithm 

$$
\begin{array}{l}
{\text { [1] Initialize } \left( \mu^{(0)}, \sigma^{2(0)}\right)} \\\\
{ \text{For s=1 to N }}: \\\\
{ \text { [2] } \mu^{(s)} \sim N\left(\frac{n \overline{Y} \sigma^{-2(s-1)}}{n \sigma^{-2(s-1)}+ \frac{1}{100}},\; \frac{1}{n \sigma^{-2(s-1)}+\frac{1}{100}}\right)}\\ 
{ \text{ [3] }  \sigma^{2(s)} \sim \text { IGamma }\left(\frac{n}{2}+1,\; \frac{1}{2} \sum_{i=1}^{n}\left(X_{i}-\mu^{(s)}\right)^{2}+5\right)} 
\end{array}
$$

<br>

(1) 초기값으로 $\mu^{(0)} = 20(분), \sigma^{2(0)} = 10(분)$로 놓고, $N = 10,000$개의 Gibbs sampler $\left(\left(\mu^{(s)}, \sigma^{2(s)}\right), s=1, \cdots, 10000\right)$를 생성해보자.

(2) 시간의 평균과 분산의 사후기댓값 추정량을 구해보자.

(3) 시간의 평균과 분산 각각의 95% Credible Interval을 구해보자.

#### Gibbs sampler

In [ ]:
# 평균 30, 표준편차 5인 정규분포를 따르는 1000개의 데이터
n = 1000
obs = norm.rvs(30, 5, n)

# 관측한 data의 histogram
plt.figure(figsize = (6, 6))
plt.hist(obs, bins = 20, )
plt.xlabel("Value")
plt.ylabel("Frequency")
plt.title("Distribution of 50 obs with $\mu=95, \sigma=1$")
plt.show()

In [ ]:
xbar = obs.mean()
xbar

In [ ]:
# setting initial values
mu_init = 20
sigma2_init = 10
N = 10000

# conditional distribution 함수 정의
def mu_given_sigma2(s2):
    # s2는 i-1번째 분산 값
    mean = (n*xbar/s2 + 0/100)/(n/s2 + 1/100)
    var = 1/(n/s2 + 1/100)
    return norm.rvs(mean, np.sqrt(var))

def sigma2_given_mu(m):
    # m은 i번째 update된 mu값
    alpha = n/2 + 1
    beta = sum((obs-m)**2)/2 + 5
    return invgamma.rvs(a = alpha, scale = beta, size = 1)

In [ ]:
### Gibbs Sampling algorithm ###
mu = []
sigma2 = []
mu_old = mu_init
sigma2_old = sigma2_init

for i in range(N):
    mu_new = mu_given_sigma2(sigma2_old)
    sigma2_new = sigma2_given_mu(mu_new)
    mu_old = mu_new
    sigma2_old = sigma2_new
    mu.append(mu_new)
    sigma2.append(sigma2_new)

In [ ]:
# 앞의 5000개는 burn-in period, 나머지 5000개로 정상분포 추정
mu_burn = mu[5000:] 
sigma2_burn = sigma2[5000:]

plt.figure(figsize = (10, 12))
plt.subplot(211)
plt.hist(np.array(mu_burn), bins = 30, )
plt.xlabel("Value")
plt.ylabel("Frequency")
plt.title("Histogram of $\mu$")

plt.subplot(212)
plt.hist(np.array(sigma2_burn), bins = 30, )
plt.xlabel("Value")
plt.ylabel("Frequency")
plt.title("Histogram of $\sigma^2$")
plt.show()

In [ ]:
# trace plot
plt.figure(figsize = (10, 10))
plt.subplot(211)
plt.plot(np.linspace(5001, 10000, 5000), np.array(mu_burn))
plt.xlabel("iter")
plt.ylabel("mu")
plt.title("Markov chain by Gibbs sampling")

plt.subplot(212)
plt.plot(np.linspace(5001, 10000, 5000), np.array(sigma2_burn))
plt.xlabel("iter")
plt.ylabel("sigma2")
plt.show()

#### Estimate

In [ ]:
# MAP estimate
print(round(np.array(mu_burn).mean(), 5)) 
print(round(np.array(sigma2_burn).mean(), 5)) 

#### Credible interval

In [ ]:
# credible interval
print(round(np.percentile(mu_burn, 2.5), 4))
print(round(np.percentile(mu_burn, 97.5), 4))

print(round(np.percentile(sigma2_burn, 2.5), 4))
print(round(np.percentile(sigma2_burn, 97.5), 4))

# Variational Inference

## Coordinate Ascent Variational Inference(CAVI)

- CAVI는 변분추론의 한 방법론으로, ELBO를 최대화하는 방향으로 학습을 진행합니다.
- 이때, **mean-field** 근사를 가정합니다. 이는 모든 잠재변수들이 서로 독립이라는 가정입니다.

\begin{equation*}
q_{\theta}(z\vert x) = \prod_{i=1}^{K}q_{\theta_i}(z_i\vert x)
\end{equation*}

- 이때, $q_{\theta_i}(z_i\vert x)$는 정규분포, 베르누이분포 등 다양한 분포를 가정합니다.
- CAVI의 알고리즘은 다음과 같이 이루어집니다.

1. Input: 혼합분포 $p(x,z)$, 관측 데이터 $x$, 잠재변수의 수 $K$
2. Initialize: $q_{i}(z_i\vert x)$ for $i=1,\cdots,K$
3. while ELBO has not conveerged do
   1. for $i=1,\cdots,K$ do
   
   \begin{equation*}
   q_{i}^{new}(z_i\vert x) \propto \exp\bigg(\mathbb{E}_{-i}\log p(x,\mathbf{z})\bigg)
   \end{equation*}
 
   2. ELBO를 계산하고, 수렴 여부를 확인합니다.

4. end
5. Output: $q_(z\vert x)=\prod_{i=1}^{K}q_{i}(z_i\vert x)$

### 예시

- 예시를 통해 CAVI를 이해해보도록 하겠습니다.
- 예시에서는 다음과 같은 $K$-Gaussian Mixture Model을 가정합니다.

\begin{equation*}
\begin{aligned}
\mu_j &\sim \mathcal{N}(0,1) \quad \text{for } j=1,\cdots,K\\
c_i &\sim \mathcal{U}(K) \quad \text{for } i=1,\cdots,N\\
x_i \vert c_i, \mu &\sim \mathcal{N}(c_i^T\mu,1) \quad \text{for } i=1,\cdots,N
\end{aligned}
\end{equation*}

- 이때, $c_i$는 $K$차원의 one-hot vector입니다.
- CAVI를 사용하기 위해, 다음과 같이 모수 $\mu, c$를 잠재변수로 보고 mean-field 근사를 가정합니다.

\begin{equation*}
q(\mu,c) = q(\mu\vert m, s^2)q(c\vert \phi) = \prod_{j=1}^{K}\mathcal{N}(\mu_j\vert m_j, s_j^2)\prod_{i=1}^{N}\mathrm{Multi}(1, \phi_i)
\end{equation*}

- 여기서 $\phi_i$는 확률 $p(c_i=j)=\phi_{ij}$로 구성된 $K$차원 벡터입니다.
- ELBO를 계산하면, 다음과 같습니다.

\begin{equation*}
\begin{aligned}
\mathrm{ELBO} &= \mathbb{E}_q\log p(x,z) - \mathbb{E}_q\log q(z)\\
&\propto \sum_j -\mathbb{E}_q\bigg[\frac{\mu_j}{2\sigma^2}\bigg] + \sum_i\sum_j \bigg[\mathbb{E}_q c_{ij} - \mathbb{E}_q\bigg[-\frac{(x_i-\mu_j)^2}{2}\bigg]\bigg] \\
&\;\;- \sum_i\sum_j \mathbb{E}_q\log\phi_{ij} + \sum_j \frac{1}{2}\log s_j^2
\end{aligned}
\end{equation*}

- CAVI의 알고리즘을 적용하면, 각 잠재변수(모수)들에 대해 다음과 같이 업데이트를 진행합니다.

1. $\phi_{ij}$

\begin{equation*}
\phi_{ij}^{new} \propto \exp\bigg(-\frac{1}{2}(m_j^2 + s_j^2) + x_i m_j\bigg)
\end{equation*}

2. $m_j$

\begin{equation*}
m_j^{new} = \frac{\sum_i\phi_{ij}x_i}{1+\sum_i\phi_{ij}}
\end{equation*}

3. $s_j^2$

\begin{equation*}
(s_j^2)^{new} = \frac{1}{1+\sum_i\phi_{ij}}
\end{equation*}

CAVI 알고리즘을 이용하여, 잠재변수의 학습이 이루어지는 과정을 살펴보도록 하겠습니다.

#### 데이터 생성하기
- 여기서는 $K=3$으로 하고, 각 성분 분포에 대해 1000개의 데이터를 생성합니다.
- `mu_true`에는 각 성분의 다음 모수가 저장됩니다.

\begin{equation*}
\mu_1 = 8.0, \quad \mu_2 = 1.2, \quad \mu_3 = -5.0
\end{equation*}

- 아래는 데이터를 생성하고, 각 성분을 구분하기 위해 색을 다르게 표시한 히스토그램입니다.

In [ ]:
K = 3
mu_true = np.array([8.0, 1.2, -5.0])
n_sample = 1000

X = np.concatenate([np.random.normal(mu, 1.0, size=(n_sample, 1)) for mu in mu_true], axis=0).ravel()
hue = np.concatenate([np.full(n_sample, i) for i in range(K)], axis=0).ravel()
df = pd.DataFrame({"x": X, "hue": hue})

plt.figure(figsize=(6, 6))
sns.histplot(data=df, x="x", hue="hue", stat="density", common_norm=False, bins=50)
plt.show()

#### CAVI 알고리즘 적용하기

CAVI 알고리즘을 적용하기 위해, 다음과 같이 `CAVI` 클래스를 정의합니다.

In [ ]:
class CAVI(object):
    def __init__(self, X, K):
        self.X = X
        self.K = K
        self.N = X.shape[0]

    def initialize(self):
        self.phi = np.random.dirichlet([1.0] * self.K, size=self.N)
        self.m = np.random.randn(self.K)
        self.s2 = np.ones(self.K) * np.random.random(self.K)
        print(f"Initial m: {self.m}, s2: {self.s2}")

    def ELBO(self):
        t1 = np.log(self.s2) - self.m
        t1 = t1.sum()
        t2 = -0.5 * np.add.outer(self.X**2, self.s2 + self.m**2)
        t2 += np.outer(self.X, self.m)
        t2 -= np.log(self.phi)
        t2 *= self.phi
        t2 = t2.sum()

        return t1 + t2

    def update_phi(self):
        t1 = np.outer(self.X, self.m)
        t2 = -0.5 * self.m**2 - 0.5 * self.s2
        exp = t1 + t2[np.newaxis, :]
        self.phi = np.exp(exp)
        self.phi /= self.phi.sum(axis=1)[:, np.newaxis]

    def update_m(self):
        self.m = (self.phi*self.X[:, np.newaxis]).sum(0) * (1 + self.phi.sum(0))**(-1)
        assert self.m.size == self.K

    def update_s2(self):
        self.s2 =(1 + self.phi.sum(0)) ** (-1)
        assert self.s2.size == self.K

- 학습을 진행하기 이전에, 만들어둔 `CAVI` 클래스의 인스턴스를 생성합니다.
- `initialize` 메소드를 통해, 잠재변수의 초기값을 설정합니다.

In [ ]:
np.random.seed(0)
cavi = CAVI(X, K)
cavi.initialize()

아래 코드는 학습 이전과 수렴 이후 모수에 대한 그래프를 그리기 위한 함수입니다.

In [ ]:
def plot(ax, i, elbo):
    ax.clear()
    sns.histplot(data=df, x="x", hue="hue", stat="density", common_norm=False, bins=50, ax=ax, palette="Set2")
    bincount = np.bincount(cavi.phi.argmax(axis=1))

    samples = np.concatenate([
        np.random.normal(cavi.m[j], np.sqrt(cavi.s2[j]), size=(bincount[j], 1)) for j in range(K)
    ]).ravel()

    sns.kdeplot(samples, ax=ax, color="black", linewidth=1.5, label="q(x)")
    ax.set_title(f"Iteration {i} : ELBO = {elbo:.2f}")
    ax.legend()
    ax.set_xlim(-10, 10)
    ax.set_ylim(0, 0.7)

#### 학습
- 학습은 다음과 같이 반복문을 적용하여 진행합니다.
- 가장 최근의 ELBO 값을 저장하여, 업데이트 이후 ELBO와의 차이가 $10^{-6}$보다 작아지면 학습을 종료합니다.

In [ ]:
last_elbo = -np.inf
fig, ax = plt.subplots(2, 3, figsize=(12, 8), sharey=True)

for i in range(100):
    if i == 0:
        plot(ax[0,0], i, cavi.ELBO()) # Initial plot

    cavi.update_phi()
    cavi.update_m()
    cavi.update_s2()

    elbo = cavi.ELBO()

    plot(ax[i//3, i%3], i, elbo)

    if elbo - last_elbo < 1e-6:
        break

    last_elbo = elbo

plt.show()

학습된 결과를 확인해보면, 비교적 적은 반복횟수로도 초기 설정한 $\mu_1, \mu_2, \mu_3$에 빠르게 수렴하는 것을 확인할 수 있습니다.

## PyMC
- `pymc` 라이브러리는 MCMC 방법론을 이용하여 베이지안 추론을 수행하는 파이썬 라이브러리입니다.
- 여기서는 `pymc` 라이브러리를 활용해 회귀분석을 진행해보고, 베이지안 신경망을 구성해보도록 하겠습니다.

### Example 1 revisited with PyMC

앞선 예시 1번의 사후분포를 PyMC를 이용하여 추정해보도록 하겠습니다.

In [ ]:
# MH algorithm

ex2 = pm.Model()

with ex2:
    # Cauchy 사전분포
    mu = pm.Cauchy('mu', 0, 1)

    # 관측값의 분포(가능도)
    y = pm.Normal('y', mu=mu, sigma=1, observed=observation) # observed 값에 관측값 벡터를 넣어준다.

    # Metropolis-Hastings 알고리즘
    trace = pm.sample(10000, tune=1000, step=pm.Metropolis())

- `chain`의 개수는 default 값인 4개로 설정되었습니다.
- Burn-in period는 1000번, Sample 10000번으로 MCMC를 진행되었고, 결과는 다음과 같습니다.

In [ ]:
# Trace plot

pm.plot_trace(trace)
plt.show()

In [ ]:
trace

In [ ]:
# MAP estimate

pm.find_MAP(model=ex2)

### Bayesian Regression with PyMC

다음과 같은 회귀직선(True model)에서 데이터를 샘플링했다고 가정합니다.
$$
Y = -2.0 + 1.2X
$$

In [ ]:
# Data Generate

n = 100
true_beta_0 = -2.0
true_beta_1 = 1.2

x = np.linspace(0,1,n)
y = true_beta_0 + true_beta_1 * x + np.random.normal(0, 0.2, n)

plt.figure(figsize=(8, 4))
plt.scatter(x, y, label='data', s=10)
plt.plot(x, true_beta_0 + true_beta_1 * x, c='r', label='$y = -2 + 1.2x$')
plt.legend()
plt.xlabel("x")
plt.ylabel("y")
plt.title("Data")
plt.show()

In [ ]:
# Bayesian Linear Regression

ex_linreg = pm.Model()

with ex_linreg:
    # 사전분포
    beta_0 = pm.Normal('beta_0', mu=0, sigma=10)
    beta_1 = pm.Normal('beta_1', mu=0, sigma=10)

    # 관측값의 분포(가능도)
    y = pm.Normal('y', mu=beta_0 + beta_1 * x, sigma=0.2, observed=y)

    # MCMC 알고리즘
    trace = pm.sample(10000, tune=1000, step=pm.Metropolis())

In [ ]:
pm.plot_trace(trace)
plt.tight_layout()
plt.show()

In [ ]:
# MAP estimate

pm.find_MAP(model=ex_linreg)

### Bayesian Logistic Regression

- `pymc` 라이브러리를 이용한 베이지안 로지스틱 회귀분석
- `iris` 데이터를 이용해 로지스틱 회귀분석을 진행해보도록 하겠습니다.

- Feature : `sepal_length`, `sepal_width`
- Class : `setosa`, `versicolor`
- 모델은 다음과 같이 주어집니다.

$$
y = \frac{1}{1+\exp(-\beta_0 - \beta_1x_1 - \beta_2x_2)}
$$

다음과 같이 uninformative prior를 가정합니다.

$$
\beta_0,\beta_1,\beta_2 \overset{iid}{\sim} \mathcal{N}(0, 10^2) \\
$$

In [ ]:
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler

iris = sns.load_dataset('iris')
iris = iris.loc[iris.species.isin(['setosa','versicolor']), ['sepal_length', 'sepal_width', 'species']]

X = iris[iris.columns.drop('species')].to_numpy()

X = StandardScaler().fit_transform(X)
y = LabelEncoder().fit_transform(iris['species'])

print(X.shape, y.shape)

주어진 데이터를 산점도로 표현하면 다음과 같습니다.

In [ ]:
# Plot the data

fig, ax = plt.subplots(1, 1, figsize=(6, 6))

sns.scatterplot(x=X[:,0], y=X[:,1], hue=y, ax=ax)
ax.set_xlabel('Sepal Length')
ax.set_ylabel('Sepal Width')
ax.legend(loc='upper right')
plt.show()

`pymc`의 `Model` 모듈을 활용하여 다음과 같이 GLM을 구성할 수 있습니다.
- Prior : 각 회귀계수에 대해($\beta_0,\beta_1,\beta_2$) 정규사전분포 부여
- Likelihood : 가능도함수(여기서는 이진 분류 문제이므로 베르누이 분포)

In [ ]:
with pm.Model() as model:
    # Priors
    intercept = pm.Normal('Intercept', 0, sigma=100)
    x1_coef = pm.Normal('sepal_length', 0, sigma=100)
    x2_coef = pm.Normal('sepal_width', 0, sigma=100)

    # Likelihood
    likelihood = pm.invlogit(intercept + x1_coef*X[:,0] + x2_coef*X[:,1]) 

    # Bernoulli random variable
    y_obs = pm.Bernoulli('y', p=likelihood, observed=y)
    
    trace = pm.sample(3000, tune=1000, chains=4)

다음과 같이 `graphviz` 모듈을 이용하여 모델의 구조를 시각화할 수 있습니다.

In [ ]:
pm.model_to_graphviz(model)

아래는 MAP 추정량을 활용한 결정경계를 그리는 코드입니다.

In [ ]:
# MAP estimate

with model:
    map_estimate = pm.find_MAP()

print(map_estimate)

# Plot the decision boundary

xs = np.linspace(-3, 3, 100)

fig, ax = plt.subplots(1, 1, figsize=(7, 7))

ax.plot(xs, (-map_estimate['Intercept'] - map_estimate['sepal_length']*xs) / map_estimate['sepal_width'], color = 'r', label='MAP estimate')
sns.scatterplot(x=X[:,0], y=X[:,1], hue=y, ax=ax)
ax.set_xlabel('Sepal Length')
ax.set_ylabel('Sepal Width')
ax.legend(loc='upper right')

plt.show()

#### Comparison with scikit-learn

- `sklearn` 라이브러리를 이용하여 로지스틱 회귀분석을 진행하고, `pymc`의 결과와 비교해보도록 하겠습니다.

In [ ]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=0).fit(X, y)

# Estimate

print(clf.intercept_, clf.coef_)

# Plot the decision boundary

fig, ax = plt.subplots(1, 1, figsize=(7, 7))

ax.plot(xs, (-clf.intercept_ - clf.coef_[0][0]*xs) / clf.coef_[0][1], color = 'b', label='Logistic Regression')
ax.plot(xs, (-map_estimate['Intercept'] - map_estimate['sepal_length']*xs) / map_estimate['sepal_width'], color = 'r', label='MAP estimate')
sns.scatterplot(x=X[:,0], y=X[:,1], hue=y, ax=ax)
plt.show()

## Bayesian Neural Network

- 일반적인 신경망 구조는 $y=f(x;W)$로 표현되며, 파라미터 $W$를 주어진 데이터로부터 학습하는 것이 목표입니다.
- 반면 베이지안 신경망은 파라미터 $W$의 사후분포를 추정하는 것이 목표입니다.

In [ ]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE

X, y = make_classification(n_samples=1000, n_features=20, n_informative=10, n_classes=2, random_state=0, n_clusters_per_class=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# plot tSNE

tsne = TSNE(n_components=2, random_state=0)
X_train_tsne = tsne.fit_transform(X_train)

plt.figure(figsize=(7, 7))
plt.scatter(X_train_tsne[y_train==0, 0], X_train_tsne[y_train==0, 1], label='Class 0', alpha=0.5)
plt.scatter(X_train_tsne[y_train==1, 0], X_train_tsne[y_train==1, 1], label='Class 1', alpha=0.5)
plt.title('tSNE')
plt.legend()
plt.show()

In [ ]:
def construct_nn(ann_input, ann_output):
    n_hidden = 32

    # Initialize random weights between each layer
    init_1 = np.random.normal(size=(X_train.shape[1], n_hidden))
    init_2 = np.random.normal(size=(n_hidden, n_hidden))
    init_out = np.random.normal(size=n_hidden)

    coords = {
        "W1": np.arange(n_hidden),
        "W2": np.arange(n_hidden),
        "features": np.arange(X_train.shape[1]),
        # "obs_id": np.arange(X_train.shape[0]),
    }
    with pm.Model(coords=coords) as bnn:
        ann_input = pm.Data("ann_input", X_train, mutable=True, dims=("obs_id", "features"))
        ann_output = pm.Data("ann_output", y_train, mutable=True, dims="obs_id")

        # Weights from input to hidden layer
        weights_in_1 = pm.Laplace(
            "w_in_1", mu=0, b=1, initval=init_1, dims=("features", "W1")
        )

        # Weights from 1st to 2nd layer
        weights_1_2 = pm.Laplace(
            "w_1_2", mu=0, b=1, initval=init_2, dims=("W1", "W2")
        )

        # Weights from hidden layer to output
        weights_2_out = pm.Normal("w_2_out", 0, sigma=1, initval=init_out, dims="W2")

        # Build neural-network using tanh activation function
        act_1 = pm.math.maximum(0,pm.math.dot(ann_input, weights_in_1))
        act_2 = pm.math.maximum(0, pm.math.dot(act_1, weights_1_2))
        act_out = pm.math.sigmoid(pm.math.dot(act_2, weights_2_out))

        # Binary classification -> Bernoulli likelihood
        out = pm.Bernoulli(
            "out",
            act_out,
            observed=ann_output,
            total_size=y_train.shape[0],  # IMPORTANT for minibatches
            dims="obs_id",
        )
    return bnn

`pm.ADVI` 클래스를 이용하면, mean-field 근사를 가정한 변분추론을 수행할 수 있습니다.

In [ ]:
BNN = construct_nn(X_train, y_train)

with BNN:
    advi = pm.ADVI()
    approx = pm.fit(30000, method=advi)

In [ ]:
# Plot ELBO
plt.figure(figsize=(6, 6))
plt.plot(approx.hist)
plt.xlabel("Iteration")
plt.ylabel("Loss")
plt.show()

Test 데이터에 대해 예측을 수행하고, 실제값과 비교해보도록 하겠습니다.

In [ ]:
with BNN:
    trace = approx.sample(1000)

# Test set

with BNN:
    pm.set_data({"ann_input": X_test, "ann_output": y_test})
    ppc = pm.sample_posterior_predictive(trace)

BNN에서의 예측은, 각 테스트 데이터에 대해 여러 사후분포 샘플과(여기서는 1000개) 여러 chain(여기서는 4개)에 대한 샘플을 이용하여, 이들의 평균을 예측값으로 사용합니다.

In [ ]:
ppc.posterior_predictive['out'].mean(('chain', 'draw'))

In [ ]:
# Accuracy

from sklearn.metrics import accuracy_score

accuracy_score(y_test, np.where(ppc.posterior_predictive['out'].mean(('chain', 'draw')) > 0.5, 1, 0))

#### Comparison with Logistic

로지스틱 회귀분석과 베이지안 신경망의 결과를 비교해보도록 하겠습니다.

In [ ]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=0).fit(X_train, y_train)
y_pred = clf.predict(X_test)

accuracy_score(y_test, y_pred)

# 0.96

정확도 비교 결과 BNN이 더 높은 정확도(99%>96%)를 보이는 것을 확인할 수 있습니다.